<a href="https://colab.research.google.com/github/st-sashou/ai_experimental/blob/master/RNN%E5%A4%9A%E5%87%BA%E5%8A%9B%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

INPUT_FILE = "/content/drive/My Drive/Colab Notebooks/ST/temperature_beverage_sales.csv"

df = pd.read_table(INPUT_FILE)
df = df.set_index('date')
df.tail()

,temperature,sports_drink,soda,bottled_water,fruit_juice
date,,,,,
2017/4/26,16.9,0.981085,0.753760,0.848250,0.747075
2017/4/27,14.7,0.854488,0.729856,0.732301,0.744117
2017/4/28,15.9,0.972824,0.944215,0.857018,0.699044
2017/4/29,16.2,0.853764,0.711245,0.875378,0.702002
2017/4/30,17.8,0.955968,0.968912,1.149194,0.575005


In [0]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
a = mms.fit_transform(df)
df_norm = pd.DataFrame(a, index=df.index, columns=df.columns)

In [0]:
import numpy as np

def generator(data, n_prev=10):
    x, y= [], []
    for i in range(len(data) - n_prev):
#         print(data.iloc[i+n_prev:i+n_prev+1, 1:5])
        x.append(data.temperature.iloc[i:i+n_prev].as_matrix()[:, np.newaxis])
        y.append(data.iloc[i+n_prev:i+n_prev+1, 1:5].as_matrix())
    return np.array(x), np.array(y)

X, Y = generator(df_norm)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)
Y_train = Y_train.transpose((2, 0, 1))
Y_test = Y_test.transpose((2, 0, 1))
Y_train = Y_train.reshape((Y_train.shape[0], Y_train.shape[1]))
Y_test = Y_test.reshape((Y_test.shape[0], Y_test.shape[1]))
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(892, 10, 1)
(4, 892)
(224, 10, 1)
(4, 224)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
X_train[0]

array([[0.10576923],
       [0.05769231],
       [0.1025641 ],
       [0.125     ],
       [0.25961538],
       [0.24679487],
       [0.23397436],
       [0.33653846],
       [0.16666667],
       [0.16987179]])

In [7]:
Y_train[0][0]

0.06164472843163325

In [8]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

HIDDEN_SIZE = 16

# 入力層、中間層
input_layer = Input(shape=(10, 1))
# x = SimpleRNN(LSTM, activation='relu', return_sequences=False, unroll=False)(x)
x = LSTM(HIDDEN_SIZE)(input_layer)

# 出力層
output_layer1 = Dense(1, activation='linear', name='output_sports_drink')(x)
output_layer2 = Dense(1, activation='linear', name='output_soda')(x)
output_layer3 = Dense(1, activation='linear', name='output_bottled_water')(x)
output_layer4 = Dense(1, activation='linear', name='output_fruit_juice')(x)

model = Model(input_layer, [output_layer1, output_layer2, output_layer3, output_layer4])
# model = Model(input_layer, output_layer1)
model.compile(optimizer='rmsprop', loss='mape')

model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 1)        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 16)           1152        input_1[0][0]                    
__________________________________________________________________________________________________
output_sports_drink (Dense)     (None, 1)            17          lstm_1[0][0]                     
__________________________________________________________________________________________________
output_soda (Dense)             (None, 1)            17          lstm_1[0][0]                     
_____________________________________

In [9]:
model.fit(X_train, Y_train.tolist(), epochs=50, batch_size=16)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
892/892 [==============================] - 1s 1ms/step - loss: 292696.9563 - output_sports_drink_loss: 81918.3903 - output_soda_loss: 83370.4603 - output_bottled_water_loss: 127357.4042 - output_fruit_juice_loss: 50.6940
Epoch 2/50
892/892 [==============================] - 0s 418us/step - loss: 202788.9710 - output_sports_drink_loss: 60360.3861 - output_soda_loss: 49528.5407 - output_bottled_water_loss: 92854.0565 - output_fruit_juice_loss: 45.9960
Epoch 3/50
892/892 [==============================] - 0s 411us/step - loss: 134421.3225 - output_sports_drink_loss: 53430.5290 - output_soda_loss: 29422.6404 - output_bottled_water_loss: 51523.3847 - output_fruit_juice_loss: 44.7660
Epoch 4/50
892/892 [==============================] - 0s 407us/step - loss: 84290.2504 - output_sports_drink_loss: 34442.3957 - output_soda_loss: 10758.1161 - output_bottled_water_loss: 39046.6269 - output_fruit_juice_loss: 43.1114
Epoch 5/50
892/892 [==

In [10]:
model.evaluate(X_test, Y_test.tolist())

224/224 [==============================] - 0s 680us/step


[7082.553022112165,
 81.65472739083427,
 120.87307739257812,
 108.69351414271763,
 6771.331451143537]

In [0]:
predicted = model.predict(X_test)

In [12]:
predicted = np.array(predicted)
predicted.shape

(4, 224, 1)

In [0]:
predicted = predicted.transpose(1, 0, 2).reshape(224, 4)

In [14]:
for x, y in zip(predicted, Y_test.T):
    print(x, y)

[ 0.07931217 -0.17673294 -0.0841073   0.5073275 ] [0.76207303 0.79674753 0.75583167 0.78101185]
[ 0.06641013 -0.15136842 -0.07186833  0.44453   ] [0.56504341 0.67965592 0.63583575 0.67271584]
[ 0.0174689  -0.02495443 -0.01048756  0.11643426] [0.07449753 0.05368874 0.11261377 0.07047799]
[ 0.04322221 -0.10255332 -0.04938421  0.32458973] [0.23896533 0.31700364 0.38887967 0.33515519]
[ 0.06337633 -0.14709507 -0.07062664  0.43661565] [0.27459793 0.42651206 0.36557462 0.2783615 ]
[0.00557955 0.00058397 0.00423031 0.0622383 ] [0.09387201 0.1324744  0.11356871 0.03633869]
[0.00378032 0.01725461 0.01303009 0.00943275] [0.0804052  0.08463165 0.10437914 0.05872554]
[ 0.01881596 -0.03121242 -0.01384726  0.13462925] [0.09731597 0.12430203 0.21143241 0.12918715]
[ 0.01321156 -0.00357423  0.00039859  0.05272874] [0.07010496 0.08345643 0.15382706 0.16745202]
[ 0.05144456 -0.12118669 -0.05854078  0.37100798] [0.39752579 0.51316068 0.55819422 0.58684741]
[ 0.01811546 -0.04012493 -0.01724898  0.17023171